In [1]:
import pandas
import requests

def handler(pd: "pipedream"):
  
  if 'pd' in locals():
    files = [
      pd.steps["trigger"]["event"]["fields"][4],
      pd.steps["trigger"]["event"]["fields"][5]];
  else:
    files = [{
      "value":[{
        "name":"download.csv",
        "url":"https://storage.googleapis.com/tally-response-assets/.../.../download.csv"
      }]
    },{
      "value":[{
        "name":"orders_export.csv",
        "url":"https://storage.googleapis.com/tally-response-assets/.../.../orders_export.csv"
      }]
    }];

  def fileCheck(fileURL, fileName, fl_dir = '/tmp/'):

    r = requests.get(fileURL);

    fl_name = '';
    fl_status = False;
    
    with open(fl_dir + fileName, 'wb') as fl:
      fl.write(r.content)

    with open(fl_dir + fileName) as fl:
      fl_line = fl.readlines();
      if '# ---' in fl_line[0]:
        fl_status = True;
      else:
        fl_status = False;

      if fl_status == True:
        fileParts = fileName.split('.')
        newName = fl_dir + fileParts[0] + 'new' + fileParts[1];
        with open(newName, 'w') as fl_new:
          for line in fl_line:
            if line.count(',') == 4:
              fl_new.write(line);
          fl_name = newName;
      else:
          fl_name = fl_dir + fileName;

    if fl_name:
      fl_df = pandas.read_csv(
        fl_name,
        delimiter=',',
        engine='python');

    return fl_df;

  def checkColumns(allCols, lst):
    if all(col in allCols for col in lst) != True:
      raise ValueError('Please make sure that the table contains the required columns; ' + ", ".join(lst));

  df_ga4 = pandas.DataFrame();
  df_shopify = pandas.DataFrame();

  for i in files:

    fileURL = i["value"][0]["url"];
    fileName = i["value"][0]["name"];

    df = fileCheck(
      fileURL = fileURL,
      fileName = fileName);

    file_name = fileName.split('.')[0];

    if file_name == 'download':
      checkColumns(df.columns, ['Date', 'Transaction ID', 'Ecommerce revenue']);
      df['Date'] = pandas.to_datetime(df['Date'], format = '%Y%m%d');

      df_ga4 = df;

    elif file_name == 'orders_export':
      cols = ['Name', 'Currency', 'Total', 'Payment Method', 'Source']; # 'Paid at', 'Fulfilled at'
      checkColumns(df.columns, cols);
      df.drop_duplicates(subset=['Name'], inplace = True);
      df_shopify = df[cols];
    else:
      raise ValueError('It is an unknown file; ' + file_name);

  multiple_transaction = df_ga4[df_ga4['Transactions'] > 1]['Transaction ID'];
  if len(multiple_transaction):
    print(str(len(multiple_transaction)) + ' transaction has multiple counts; ' + ', '.join(multiple_transaction));

  multiple_event_count = df_ga4[df_ga4['Event count'] > 1]['Transaction ID'];
  if len(multiple_event_count):
    print(str(len(multiple_event_count)) + ' transaction has multiple counts; ' + ', '.join(multiple_transaction));

  df_shopify = df_shopify.astype({
    'Source': 'category',
    'Payment Method': 'category',
    'Currency': 'category'});

  data = pandas.merge(left = df_shopify,
                    right = df_ga4,
                    left_on = ['Name', 'Total'],
                    right_on = ['Transaction ID', 'Ecommerce revenue'],
                    how = 'left')

  return data.to_dict()

ModuleNotFoundError: No module named 'pandas'